In [26]:
# TODO support layers, units
import ray
import dni
import importlib
importlib.reload(dni)
import numpy as np
# ray.shutdown()
# ray.init(address="10.128.0.51:6379")

In [4]:
@ray.remote
class Scanner(dni.access.Scan):
    def __init__(self):
        self.f = open('sqldata.txt', 'r')
        self.f.seek(0)
        self.nextline = self.f.readline()
    def Next(self):
        if not self.HasMore():
            return ""
        currentline = self.nextline
        self.nextline = self.f.readline()
        return self.postprocess(currentline)

    def HasMore(self):
        if self.nextline:
            return True
        else:
            return False
    def postprocess(self,sttr):
        return sttr.split("\n")[0]

@ray.remote
class ActivationExt(dni.model.Extractor):
    def __init__(self, modelname, layer):
        from keras.models import Model, load_model
        newmodel = load_model(modelname)
        newmodel._make_predict_function()
        outputs = [newmodel.layers[l].output for l in layer]
        self.model = Model(inputs = newmodel.input, outputs = outputs)
    def extract(self,input,unit):
        input = self.preprocess(input)
        pred = self.model.predict(input)
        if not isinstance(pred, list):
            pred = [pred]
        for i in range(len(pred)):
            pred[i] = pred[i][...,unit]
        return pred
    def preprocess(self, input):
        seq = input
        char2int = {' ': 72, '$': 23, "'": 12, '(': 68, ')': 8, '*': 71, '+': 33, ',': 24, '-': 26, '.': 2,
         '/': 5, '0': 22, '1': 6, '2': 43, '3': 38, '4': 14, '5': 58, '6': 66, '7': 18, '8': 55, '9': 69,
         '<': 52, '=': 57, '>': 60, 'A': 1, 'B': 47, 'C': 9, 'D': 40, 'E': 62, 'F': 59, 'G': 21, 'H': 32,
         'I': 70, 'J': 56, 'L': 15, 'M': 50, 'N': 63, 'O': 7, 'P': 31, 'R': 27, 'S': 39, 'T': 51, 'U': 25,
         'V': 29, 'W': 49, 'Y': 20, 'a': 61, 'b': 64, 'c': 44, 'd': 37, 'e': 16, 'f': 54, 'g': 11, 'h': 28,
         'i': 67, 'j': 17, 'k': 46, 'l': 3, 'm': 13, 'n': 42, 'o': 65, 'p': 41, 'q': 34, 'r': 10, 's': 19, 
        't': 36, 'u': 48, 'v': 53, 'w': 30, 'x': 4, 'y': 45, 'z': 35, '~': 0}
        pad_char='~'
        w_size = 30
        step_size = 5
        n_test_tuples = 128
        if w_size == -1:
            w_size = max(len(S) for S in sequences)
        X = []
        y = []

        steps = range(step_size, len(seq), step_size)
        if len(steps) == 0:
            steps = [len(seq)-1]
        for i in steps:
            end = i
            start = max(0, i-w_size)
            s = seq[start:end]
            assert len(s) <= w_size
            x = s.rjust(w_size, pad_char)
            assert len(x) == w_size
            X.append(x)
            y.append(seq[i])
            if len(X) > n_test_tuples:
                break

        test_from = X
        X_test, char2int = dni.tool.TwoDimEncoders.raw_to_bin_tensor(test_from, cust_char2int=char2int)
        return X_test
    
def generate_parsetree(inputdata):
    import warnings
    import copy
    import collections
    import time, timeit
    import random
    import numpy as np
    import nltk
    from nltk import Tree
    from nltk import PCFG,CFG
    from nltk.grammar import is_terminal, is_nonterminal, ProbabilisticProduction, Nonterminal
    from nltk.probability import DictionaryProbDist
    expr = inputdata
    gram = None
    with open('sql_full_XL.pcfg', 'r') as f:
        pcfg_string = f.read()
        from nltk import PCFG
        gram = PCFG.fromstring(pcfg_string)
    rules = [r for r in gram.productions() if r.prob() > 0]
    rule_index = collections.defaultdict(list)
    for r in rules:
        rule_index[r.lhs()].append(r)
    ix = rule_index
    len_ix = {lhs:len(rules) for lhs,rules in ix.items()}
    counter = 0
    term_index = {}
    new_rules = []
    for rule in gram.productions():
        # If Nonterm := Term or Nontern := Nonterm rule, skip
        if len_ix[rule.lhs()]==1 and len(rule.rhs())==1:
            new_rules.append(rule)
            continue
        # Otherwise creates a new rule
        new_rhs = []
        for r in rule.rhs():
            if is_nonterminal(r) :
                new_rhs.append(r)
            else:
                if r not in term_index:
                    new_left = Nonterminal('symb_'+str(r))
                    prule = ProbabilisticProduction(new_left,
                                                    [r],
                                                    prob=1.0)
                    term_index[r]=prule
                new_rhs.append(new_left)
        new_rule = ProbabilisticProduction(rule.lhs(),
                                           new_rhs,
                                           prob=rule.prob())
        new_rules.append(new_rule)
    new_rules += term_index.values()
    gram = PCFG(gram.start(), new_rules)
    rules = [r for r in gram.productions() if r.prob() > 0]
    rule_index = collections.defaultdict(list)
    for r in rules:
        rule_index[r.lhs()].append(r)
    old_grammar = gram
    # BFS
    arules = set()
    visited = set()
    Q = collections.deque()
    Q.append(gram.start())
    while len(Q) > 0:
        # Gets first symbol in queue
        symb = Q.popleft()
        # Fetches corresponding rules
        rules = rule_index[symb]
        for r in rules:
            arules.add(r)
            for symb in r.rhs():
                if is_nonterminal(symb) and not symb in visited:
                    Q.append(symb)
                    visited.add(symb)
    # Creates new rules
    arules = list(arules)
    gram = PCFG(old_grammar.start(), arules)

    symbols = set()
    for rule in gram.productions():
        symbols.add(rule.lhs())
        for r in rule.rhs():
            symbols.add(r)

    
    term_nodes = [S for S in symbols if is_terminal(S)]
    terminals = set(term_nodes)
    L = max(len(w) for w in terminals)
    def terminal(i, expr):
        for wid in reversed(range(1, L+1)):
            j = i + wid
            if j > len(expr):
                continue
            w = expr[i:j]
            if w in terminals:
                return w
        return None

    tokens = []
    index = []
    i = 0
    skipchars=['~']
    while i < len(expr):
        if not expr[i] in skipchars:
            term = terminal(i, expr)
            if term is None:
                if not ignore_errors:
                    raise ValueError('Could not match token', expr[i:])
                else:
                    print("null")
            tokens.append(term)
            index.append((i,len(term)))
            i += len(term)
        else:
            i += 1
    parser = nltk.EarleyChartParser(gram)
    parse = None
    for p in parser.parse(tokens):
        parse = p
        break

    rules = [r for r in gram.productions() if r.prob() > 0]
    rule_index = collections.defaultdict(list)
    for r in rules:
        rule_index[r.lhs()].append(r)
    # Gets the symbols BFS
    visited = [gram.start()]
    Q = collections.deque()
    Q.append(gram.start())

    while len(Q) > 0:
        # Gets first symbol in queue
        symb = Q.popleft()
        # Fetches corresponding rules
        rules = rule_index[symb]
        for r in rules:
            for symb in r.rhs():
                if is_nonterminal(symb) and not symb in visited:
                    Q.append(symb)
                    visited.append(symb)

    visited = list(reversed(visited))

    # Alternative method for checking correctness
    alt_visited = [S for S in symbols if is_nonterminal(S)]
    assert set(visited) == set(alt_visited)

    non_term_nodes = visited
    tree = parse
    n_tokens = len(tree.leaves())
    nt_symbols = non_term_nodes

    rule_feats = np.zeros((n_tokens, len(nt_symbols)))
    rule2feat = {s.symbol():i for i,s in enumerate(nt_symbols)}

    def visit(tree, offset):
        if isinstance(tree, Tree):
            n_tokens = 0
            for subtree in tree:
                n_tokens += visit(subtree, offset + n_tokens)
            symb = tree.label()
            j = rule2feat[symb]
            rule_feats[offset:offset+n_tokens, j] = 1
            return n_tokens
        else:
            return 1

    visit(tree, 0)
    seq = expr
    lex_index = index
    w_feats = rule_feats


    ch_len = len(seq)
    w_len  = w_feats.shape[0]
    n_feats = w_feats.shape[1]
    ch_feats = np.zeros((ch_len, n_feats))

    for i_w in range(w_len):
        f_w  = w_feats[i_w,...]
        i_ch,len_ch = lex_index[i_w]
        f_ch = np.tile(f_w, len_ch).reshape((len_ch, n_feats))
        ch_feats[i_ch:i_ch+len_ch,:] = f_ch
    return [ch_feats,nt_symbols]

# feature for "where" symbol in sql
def extractfeature1(intermiediate, inputdata):  
    import numpy as np
    tree = intermiediate[0]
    names = intermiediate[1]
    seq= inputdata
    pad_char='~'
    w_size = 30
    step_size = 5
    if w_size == -1:
        w_size = max(len(S) for S in sequences)
    X = []
    y = []
    prov_index = {}

    steps = range(step_size, len(seq), step_size)
    if len(steps) == 0:
        steps = [len(seq)-1]
    for i in steps:
        end = i
        start = max(0, i-w_size)
        s = seq[start:end]
        assert len(s) <= w_size
        x = s.rjust(w_size, pad_char)
        assert len(x) == w_size
        X.append(x)
        y.append(seq[i])
        prov_index[x] = (start, end)
    featuress = []
    for idx, name in enumerate(names):
        features = []
        for key, value in prov_index.items(): 
            start, end = value
            feature = tree[start:end,idx]
            L = end - start
            if L < 30:
                tmp = np.zeros((30))
                tmp[-L:] = feature
                feature = tmp
            features.append(feature)
        feat_name = 'F_' + name.symbol()
        featuress.append(features)
    return np.array(featuress[119])

# feature for "From" symbol in sql
def extractfeature2(intermiediate, inputdata):  
    import numpy as np
    tree = intermiediate[0]
    names = intermiediate[1]
    seq= inputdata
    pad_char='~'
    w_size = 30
    step_size = 5
    if w_size == -1:
        w_size = max(len(S) for S in sequences)
    X = []
    y = []
    prov_index = {}

    steps = range(step_size, len(seq), step_size)
    if len(steps) == 0:
        steps = [len(seq)-1]
    for i in steps:
        end = i
        start = max(0, i-w_size)
        s = seq[start:end]
        assert len(s) <= w_size
        x = s.rjust(w_size, pad_char)
        assert len(x) == w_size
        X.append(x)
        y.append(seq[i])
        prov_index[x] = (start, end)
    featuress = []
    for idx, name in enumerate(names):
        features = []
        for key, value in prov_index.items(): 
            start, end = value
            feature = tree[start:end,idx]
            L = end - start
            if L < 30:
                tmp = np.zeros((30))
                tmp[-L:] = feature
                feature = tmp
            features.append(feature)
        feat_name = 'F_' + name.symbol()
        featuress.append(features)
    return np.array(featuress[96])



In [35]:
clusterid = "10.128.0.51:6379"
AccessMethod= Scanner
ActivationExt = ActivationExt
Neuron = [[[0],[0,1,2,3]]]
Models = ["track_history/models-03-2.78.hdf5"]
FeaturesFunctions = [[[extractfeature1,extractfeature2],generate_parsetree]]
FeatureNames = ["WHERE","FROM"]
MetricName = ["Correlation"]

dni.inspect(clusterid = "10.128.0.51:6379",
            AccessMethod= Scanner,
            ActivationExt = Act_extract, Neuron = [[[0],[0,1,2,3]]], 
            Models = ["track_history/models-03-2.78.hdf5"],
            FeaturesFunctions = [[[extractfeature1,extractfeature2],generate_parsetree]],
            FeatureNames = ["WHERE","FROM"],
            MetricName = ["Correlation"])

In [44]:
layers = []
units = []
ray.shutdown()
ray.init(address=clusterid)
for i in range(len(Neuron)):
    layers.append(Neuron[i][0])
    units.append(Neuron[i][1])
    
maxlayer = 0
maxunits = 0
for i in range(len(Models)):
    maxlayer = max(maxlayer,len(layers[i]))
    maxunits = max(maxunits,len(units[i]))

numfeature = 0
for i in range(len(FeaturesFunctions)):
    if type(FeaturesFunctions[i]) is list and len(FeaturesFunctions[i]) == 2:
        numfeature += len(FeaturesFunctions[i][0])
    else:
        numfeature += 1

metrics = [None]*len(MetricName)
for i in range(len(MetricName)):
    if MetricName[i] == "Correlation":
        metrics[i] = dni.metric.Batch_Incremental_Comprehensive_Correlation(len(Models),maxlayer,maxunits,numfeature)

ModelActor = ActivationExt.remote(Models[0],layers[0])

FeatActor = [None]*len(FeaturesFunctions)
for i in range(len(FeaturesFunctions)):
    # if has intermediate result
    if type(FeaturesFunctions[i]) is list and len(FeaturesFunctions[i]) == 2:
        featureclass = ray.remote(dni.feature.CachedFeatureExtractor)
        FeatActor[i] = featureclass.remote(FeaturesFunctions[i][0],FeaturesFunctions[i][1])
    else:
        FeatActor[i] = FeatureExtractor[i].remote(FeaturesFunctions[i],FeatureNames[i])

ScannerActor = AccessMethod.remote()
act = []
feature = [[] for i in range(len(FeatureNames))]
while ray.get(ScannerActor.HasMore.remote()):
    data = ScannerActor.Next.remote()
    Ext = ModelActor.extract.remote(data,units[0])
    act.append(ray.get(Ext))
    currf = 0
    # for each group of feature functions
    for i in range(len(FeaturesFunctions)):
        if type(FeaturesFunctions[i]) is list and len(FeaturesFunctions[i]) == 2:
            FeatActor[i].setup.remote(data)
            # for each feature functions
            for j in range(len(FeaturesFunctions[i][0])):
                FeatActor[i].extract.remote(data,1)
                feature = ray.get(FeatActor[i].extract.remote(data,j))
                # for each layer
                for k in range(len(layers[0])):
                    # for each units
                    for l in range(len(units[i])):
                        # for each metric
                        for m in range(len(MetricName)):
                            metrics[i].increment(feature,ray.get(Ext)[k][...,l],0,k,l,currf)
                currf += 1
        else:
            feature[currf].append(ray.get(FeatActor[i].extract.remote(data)))
            currf += 1

from prettytable import PrettyTable
t = PrettyTable(['Model','Metric','Feature','Neuron','Score'])

#for each model
for i in range(len(Models)):
    # for each layer
    for j in range(len(layers[i])):
        # for each unit
        for k in range(len(units[i])):
            # for each feature
            for l in range(numfeature):
                t.add_row([Models[i],MetricName[j],FeatureNames[l],"("+str(layers[i][j])+","+str(units[i][k])+")",metrics[i].extract(0,j,k,l)])
                
print(t)


2019-11-11 20:04:38,109	WARNING worker.py:1268 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-11-11 20:04:38,142	WARNING actor.py:583 -- Actor is garbage collected in the wrong driver. Actor id = ActorID(5d29414501000000), class name = Scanner.


(pid=3356) Using TensorFlow backend.
(pid=3356) WARNING:tensorflow:From /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(pid=3356) Instructions for updating:
(pid=3356) If using Keras pass *_constraint arguments to layers.
(pid=3356) 2019-11-11 20:04:42.698390: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  AVX2 FMA
(pid=3356) To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=3356) 2019-11-11 20:04:42.707196: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
(pid=3356) 2019-11-11 20:04:42.707546: I tensorflow/compiler/xla/service/service.cc:168] XLA servic

(pid=2004) Using TensorFlow backend.
(pid=2004) WARNING:tensorflow:From /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(pid=2004) Instructions for updating:
(pid=2004) If using Keras pass *_constraint arguments to layers.
(pid=2004) 2019-11-11 17:48:37.846780: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  AVX2 FMA
(pid=2004) To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=2004) 2019-11-11 17:48:38.353038: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
(pid=2004) 2019-11-11 17:48:38.353405: I tensorflow/compiler/xla/service/service.cc:168] XLA servic

In [ ]:
m = 

In [291]:
metric = []
#for each model
for i in range(len(Models)):
    metric_temp1 = []
    # for each layer
    for j in range(len(layers[i])):
        # for each unit
        for k in range(len(units[i])):
            # for each feature
            for l in range(len(FeatureNames)):
                
                
    
                

[Actor(CachedFeatureExtractor, c53598d002000000)]

1 4


In [316]:
# for each load of input data, each layer, get a number of activation, the last dimension is unit
act[0][0].shape

(54, 30, 4)

In [315]:
# for each feature, each load of input data, get a number of features
feature[0][0].shape

(54, 30)

+-----------------------------------+-------------+---------+--------+-----------------------+
|               Model               |    Metric   | Feature | Neuron |         Score         |
+-----------------------------------+-------------+---------+--------+-----------------------+
| track_history/models-03-2.78.hdf5 | Correlation |  WHERE  | (0,0)  |  0.02771628520146099  |
| track_history/models-03-2.78.hdf5 | Correlation |   FROM  | (0,0)  |  0.11961696236126476  |
| track_history/models-03-2.78.hdf5 | Correlation |  WHERE  | (0,1)  |  0.05012749261812874  |
| track_history/models-03-2.78.hdf5 | Correlation |   FROM  | (0,1)  |  0.12826532216877126  |
| track_history/models-03-2.78.hdf5 | Correlation |  WHERE  | (0,2)  |  -0.04503853262076593 |
| track_history/models-03-2.78.hdf5 | Correlation |   FROM  | (0,2)  | -0.038613080219511146 |
| track_history/models-03-2.78.hdf5 | Correlation |  WHERE  | (0,3)  |  0.059537094383411385 |
| track_history/models-03-2.78.hdf5 | Correlation 

In [ ]:



print(t)

In [229]:
s = Scanner.remote()

In [138]:
ray.get(s.HasMore.remote())

True

In [230]:
lines  = []
while ray.get(s.HasMore.remote()):
    lines.append(s.Next.remote())
ray.get(lines[0])

' SELECT *, ix.* FROM )  SELECT pqohm.nl65p  AS bpxhg FROM yc11 WHERE p65.bn7<oo  AND ..false..<>.fj. ORDER BY ff DESC ) AS r08z union  SELECT .1875. * t8a5gu, false, wtl.* FROM vz AS rk4j6.* q36 AS w2orpr INNER JOIN l5  ON q3<=to4uao.tc WHERE f9c7kr<=azf8x GROUP BY  in7kmt$'

In [145]:
line1 = ray.get(lines[0])

In [225]:
intermediate = generate_parsetree(ray.get(lines[1]))
intermediate[0].shape

(130, 171)

In [241]:
# for i,j in enumerate(intermediate[1]):
#     print(i,j)

In [240]:
f = extractfeature1(intermediate,ray.get(lines[1]))
f.shape

(25, 30)

In [256]:
featureext = ray.remote(dni.feature.CachedFeatureExtractor)

In [257]:
featureext = featureext.remote([extractfeature1,extractfeature2],generate_parsetree)

In [258]:
featureext.setup.remote(ray.get(lines[0]))
ray.get(featureext.extract.remote(lines[0],1))

ObjectID(ea7106654faf4b49c2340100000000c001000000)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

SyntaxError: can't assign to literal (<ipython-input-260-54a76c4220d1>, line 1)

In [ ]:
Neuron = [[[0],[0,1,2,3]]]
Models = ["track_history/models-03-2.78.hdf5"]
clusterid = "10.128.0.51:6379"
AccessMethod= Scanner
ActivationExt = ActivationExt
Models = ["Models/boolean.h5"]
FeaturesFunctions = features 
FeatureNames = feature_names
FeatureExtractor = [F_Extractor]*6
MetricExtractor = [corr_metric] MetricName = Metric_names

In [177]:

layers = []
units = []
for i in range(len(Neuron)):
    layers.append(Neuron[i][0])
    units.append(Neuron[i][1])

In [186]:
ModelActor = ActivationExt.remote(Models[0],layers[0])
act1 = ray.get(ModelActor.extract.remote(lines[1],units[0]))
act1[0].shape

2019-11-10 20:16:22,983	WARNING worker.py:1619 -- The actor or task with ID ffffffffffffd1cda02401000000 is pending and cannot currently be scheduled. It requires {} for execution and {CPU: 1.000000} for placement, but this node only has remaining {object_store_memory: 2.832031 GiB}, {CPU: 4.000000}, {memory: 8.251953 GiB}. In total there are 0 pending tasks and 1 pending actors on this node. This is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increase the resources available to this Ray cluster.


(pid=18136) Using TensorFlow backend.
(pid=18136) WARNING:tensorflow:From /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(pid=18136) Instructions for updating:
(pid=18136) If using Keras pass *_constraint arguments to layers.
(pid=18136) 2019-11-10 20:16:25.174985: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  AVX2 FMA
(pid=18136) To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=18136) 2019-11-10 20:16:25.182514: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
(pid=18136) 2019-11-10 20:16:25.182985: I tensorflow/compiler/xla/service/service.cc:168] XL

(25, 30, 4)